In [ ]:
%%capture
!pip install langchain==0.1.1 openai==1.8.0 langchain-openai duckduckgo-search wikipedia langchainhub faiss-cpu
pip install duckduckgo_search numexpr wikipedia
!pip install faiss-cpu
pip install langchainhub

In [1]:
import os
import getpass

In [2]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

# 🤖 Agents in LangChain

**Agents** are the driving force in LangChain, smart systems that use language models to work with other digital tools, perfect for tasks like Q&A, API interactions, and more.

### 🆚 Agents vs. Chains

- **Agents** dynamically decide on actions using language models.
- **Chains** have a predetermined sequence of actions.

Agents are about smart decision-making and interacting with the digital environment, while chains are about a pre-set flow of information.

### 🛠️ Tools and Toolkits

- **Tools** are like an agent's skills, specific to tasks like data retrieval.
- **Toolkits** bundle these skills for more complex tasks.

### 🧠 The Role of Memory

- Memory gives agents the ability to remember and use past interactions, making them smarter over time.



In [14]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain_openai import ChatOpenAI

# instantiate the llm
llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.0)

# Giving the Agent Tools


 - 🛠️ 🔧 Tools are integrated into agents either all at once during setup  or individually as needed .

 - 🧰 Opt for LangChain's ready-made tools or craft your own custom tools to meet your unique needs.

You can find the available native tools [here](https://github.com/langchain-ai/langchain/blob/ccb9e3ee2d4ffde1bb33c6c0df0db87aff3341bf/libs/langchain/langchain/agents/load_tools.py#L409) and look at the dictionary `_EXTRA_OPTIONAL_TOOLS` for the key of the tool.

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 5.5 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=82fbd596fc07409ea2bc62e1d8efce1b6784a8159f13a51acce9b4fd97dad24b
  Stored in directory: /home/vscode/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.


In [16]:
# equip it with some tools
tools = load_tools(["ddg-search", "llm-math", "wikipedia"], llm=llm)

# Initialize the agent

**Setting Up an Agent:**

There are two main methods to bring your LangChain agent to life:

1. **AgentExecutor** 🛠️: Acts as the agent's "command center." It's where the agent's actions are executed and where its memory is managed. You'll need an agent, a toolkit, and an optional memory component to get started.

2. **Initialize Agent** 🚀: This is the streamlined method for a quick launch. Provide the agent class, language model, and any tools you want to use, and it sets everything up for you.



In [17]:
query = """
Who is the current Chief AI Scientist at Meta AI? When was he born? Where was he born? What's the current temperature there?
"""

In [18]:
# initialize the agent
agent = initialize_agent(tools,
                         llm,
                         agent="zero-shot-react-description",
                         verbose=True)

agent.run(query)



> Entering new AgentExecutor chain...
I should use Wikipedia to find information about the current Chief AI Scientist at Meta AI and then use DuckDuckGo Search to find the current temperature in his birthplace.
Action: wikipedia
Action Input: Chief AI Scientist at Meta AI
Observation: Page: Meta AI
Summary: Meta AI is an artificial intelligence laboratory owned by Meta Platforms Inc. (formerly known as Facebook, Inc.). Meta AI develops various forms of artificial intelligence, including augmented and artificial reality technologies. Meta AI is also an academic research laboratory focused on generating knowledge for the AI community. This is in contrast to Facebook's Applied Machine Learning (AML) team, which focuses on practical applications of its products. 



Page: Mistral AI
Summary: Mistral AI is a French company selling artificial intelligence (AI) products. It was founded in April 2023 by previous employees of Meta Platforms and Google DeepMind. The company raised €385 million

'Yann LeCun is the current Chief AI Scientist at Meta AI. He was born on July 8, 1960, in France. The current temperature in his birthplace is not available in the search results.'

### AgentExecutor

The `AgentExecutor` class is responsible for executing the agent's actions and managing the agent's memory.

It takes an agent, a set of tools, and an optional memory object as input.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [25]:
from langchain.agents import AgentExecutor
from langchain.agents import create_openai_functions_agent
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")

llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.0)

tools = load_tools(["ddg-search", "llm-math", "wikipedia"], llm=llm)

agent = create_openai_functions_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True)

In [26]:
query = "What's the current weather in Winnipeg?"

agent_executor.invoke({"input":query})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'current weather in Winnipeg'}`


2024-04-29. Current conditions and forecasts including 7 day outlook, daily high/low temperature, warnings, chance of precipitation, pressure, humidity/wind chill (when applicable) historical data, normals, record values and sunrise/sunset times. 2024-04-29. Detailed forecast for the next 24 hours - temperature, weather conditions, likelihood of precipitation and winds. Winnipeg Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the Winnipeg area. The current weather conditions in Winnipeg show an average temperature of around -10°C in winter, with occasional extreme cold spells and snowfall. During the summer months, temperatures can reach up to 20°C and higher, with occasional thunderstorms. The best source for accurate and up-to-date information on Winnipeg weather conditions

{'input': "What's the current weather in Winnipeg?",
 'output': 'The current weather in Winnipeg is -14°C with a gentle breeze from the north. It feels like -22°C. There is no precipitation at the moment.'}

# 🔄 **Agents and Memory**

### **Stateless Agent Characteristics**

- **Current Limitation**: Inability to recall past interactions by default.

### **Enabling Memory**:

- **Method**: Integrate previous chat history into the agent.

- **Key Requirement**: The chat history variable must be named "chat_history" for compatibility with the current prompt.

- **Flexibility**: Altering the prompt
allows for different variable naming.

🔑 **Takeaway**: Adjusting the prompt or incorporating chat history enables stateful interactions, enhancing the agent's capability to remember and utilize past interactions.

In [27]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [28]:
message_history = ChatMessageHistory()

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [29]:
agent_with_chat_history.invoke(
    {"input": query},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'current weather in Winnipeg'}`


Get the current weather conditions and outlook for Winnipeg, MB from Environment Canada. See the temperature, wind, humidity, pressure, visibility, and UV index for today and the next seven days. Winnipeg Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the Winnipeg area. The current weather conditions in Winnipeg show an average temperature of around -10°C in winter, with occasional extreme cold spells and snowfall. During the summer months, temperatures can reach up to 20°C and higher, with occasional thunderstorms. The best source for accurate and up-to-date information on Winnipeg weather conditions is ... Providing a local hourly Winnipeg weather forecast of rain, sun, wind, humidity and temperature. The Long-range 12 day forecast also includes detail for Winnipeg weather

{'input': "What's the current weather in Winnipeg?",
 'chat_history': [],
 'output': 'The current weather in Winnipeg is -14°C with a gentle breeze from the north. The temperature feels like -22°C. There is no precipitation at the moment.'}

In [30]:
agent_with_chat_history.invoke(
    {"input": "What do you reckon I should wear then?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
I recommend dressing warmly for the current weather in Winnipeg. Here are some suggestions:

1. Layer up with a thermal shirt, sweater, and a heavy winter coat.
2. Wear insulated pants or jeans with thermal leggings underneath.
3. Don't forget to wear a hat, gloves, and a scarf to protect your head, hands, and neck from the cold.
4. Choose warm and waterproof boots to keep your feet dry and insulated.

Stay warm and cozy!

> Finished chain.


{'input': 'What do you reckon I should wear then?',
 'chat_history': [HumanMessage(content="What's the current weather in Winnipeg?"),
  AIMessage(content='The current weather in Winnipeg is -14°C with a gentle breeze from the north. The temperature feels like -22°C. There is no precipitation at the moment.')],
 'output': "I recommend dressing warmly for the current weather in Winnipeg. Here are some suggestions:\n\n1. Layer up with a thermal shirt, sweater, and a heavy winter coat.\n2. Wear insulated pants or jeans with thermal leggings underneath.\n3. Don't forget to wear a hat, gloves, and a scarf to protect your head, hands, and neck from the cold.\n4. Choose warm and waterproof boots to keep your feet dry and insulated.\n\nStay warm and cozy!"}

# Creating agents in LangChain v0.2 and beyond

Starting in v0.2 LangChain will stop using `initialize_agent` in favor of individual agent methods, as these are more clear.

You can see the affected agents in the [changelog](https://python.langchain.com/docs/changelog/langchain).

In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [46]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.0)

Defaulting to user installation because normal site-packages is not writeable


In [48]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256).split_documents(docs)

vector = FAISS.from_documents(documents, OpenAIEmbeddings())

retriever = vector.as_retriever()

In [49]:
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

tools = [search, retriever_tool]

In [50]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [51]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [52]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [53]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


[Document(page_content='Getting started with LangSmith | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Getting started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'Introduction', 'language': 'en'}), Document(page_content="Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookQuick StartOn this pageGetting started with LangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!Install LangSmith\u200bWe offer Python and Typescript SDKs for all your LangSmith needs.PythonTypeScriptpip install 

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith is a platform for building production-grade LLM applications that allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. It offers Python and Typescript SDKs for all your LangSmith needs. You can create an API key, set up your environment, and log your first trace using LangSmith. Additionally, LangSmith provides capabilities for tracing, evaluation, production monitoring, and automations.\n\nFor more information on how LangSmith can help with testing, you can refer to the [LangSmith documentation](https://docs.smith.langchain.com/overview).'}